# Class IV - An introduction to GasModels.jl

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of the [GasModels.jl](https://github.com/lanl-ansi/GasModels.jl) package. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) and [Class II - An introduction  to JuMP](Class%20II%20-%20An%20introduction%20to%20JuMP.ipynb) first.

As in Class II, run the following magic sauce to check we're good to go.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Installed Widgets ─────────────── v0.4.4
 Installed Measures ────────────── v0.3.0
 Installed RecipesBase ─────────── v0.6.0
 Installed Requires ────────────── v0.5.2
 Installed InteractBulma ───────── v0.4.2
 Installed Pidfile ─────────────── v1.1.0
 Installed Interact ────────────── v0.9.0
 Installed GLPK ────────────────── v0.9.1
 Installed CSSUtil ─────────────── v0.1.0
 Installed FixedPointNumbers ───── v0.5.3
 Installed Showoff ─────────────── v0.2.1
 Installed PlotThemes ──────────── v0.3.0
 Installed FunctionalCollections ─ v0.5.0
 Installed LinQuadOptInterface ─── v0.6.0
 Installed ColorTypes ──────────── v0.7.5
 Installed AssetRegistry ───────── v0.1.0
 Installed GR ──────────────────── v0.36.0
 Installed Reexport ────────────── v0.2.0
 Installed PlotUtils ───────────── v0.5.5
 Installed Pl

### Background 

This [presentation](https://github.com/lanl-ansi/tutorial-grid-science-2019/blob/master/assets/infrastructure_optimization_in_julia.pdf) provides motivation and context for this notebook.

Some additional informaiton is also available at,
* [GasModels Documentation](https://lanl-ansi.github.io/GasModels.jl/stable/)


### Working with the Network Model

Two 40-node gas network models are provided with this tutorial.  The MATLAB-like input data files can be viewed [here](http://localhost:8888/edit/data/gaslib40.m) and [here](http://localhost:8888/edit/data/gaslib40-ls.m).

Similar to PowerModels, the `parse_file` function from GasModels is used to load a text files into the GasModels data model, 

In [2]:
using GasModels

data = GasModels.parse_file("data/gaslib40.m")

[warn | InfrastructureModels]: Matlab parser skipping the following line:
  end


┌ Warning: no case version found in .m file.  The file seems to be missing "mgc.version = ..."
└ @ GasModels /Users/kaarthiksundar/.julia/packages/GasModels/edHge/src/io/matlab.jl:111


Dict{String,Any} with 19 entries:
  "gas_specific_gravity"         => 0.6
  "temperature"                  => 273.15
  "source_type"                  => ".m"
  "name"                         => "gaslib40"
  "source_version"               => "0.0.0+"
  "multinetwork"                 => false
  "gas_molar_mass"               => 0.0185674
  "consumer"                     => Dict{String,Any}("24"=>Dict{String,Any}("ql…
  "standard_density"             => 1.0
  "baseQ"                        => 604.167
  "compressibility_factor"       => 0.8
  "specific_heat_capacity_ratio" => 1.4
  "producer"                     => Dict{String,Any}("1"=>Dict{String,Any}("qgm…
  "per_unit"                     => true
  "junction"                     => Dict{String,Any}("32"=>Dict{String,Any}("ju…
  "baseP"                        => 8101325
  "sound_speed"                  => 312.805
  "R"                            => 8.314
  "connection"                   => Dict{String,Any}("32"=>Dict{String,Any}("le…

In this case the file parser generated a warning messages.  

The second warning block message indicates that the data file did not have any version information. Still the parser will load this data.

Now, lets look at some of the network data.  The data for the first bus, load and branch can be viewed as follows,

In [3]:
data["connection"]["1"]

Dict{String,Any} with 8 entries:
  "length"          => 76893.6
  "f_junction"      => 32
  "friction_factor" => 0.0074
  "status"          => 1
  "t_junction"      => 18
  "diameter"        => 0.8
  "type"            => "pipe"
  "index"           => 1

In [4]:
data["connection"]["100000"]

Dict{String,Any} with 10 entries:
  "c_ratio_max" => 5.0
  "f_junction"  => 33
  "qmax"        => 700.0
  "status"      => 1
  "t_junction"  => 100021
  "c_ratio_min" => 1.0
  "power_max"   => 3500.0
  "qmin"        => 0.0
  "type"        => "compressor"
  "index"       => 100000

Here, connection can either be of type "pipe" in which case it has a length, diameter, and friction_factor associated with it or can be a compressor with a different set of fields as shown above. 

In [5]:
data["junction"]["1"]

Dict{String,Any} with 7 entries:
  "junction_type" => 0
  "status"        => 1
  "junction_i"    => 1
  "pmax"          => 1.0
  "p_nominal"     => 3.10133e6
  "pmin"          => 0.382817
  "index"         => 1

That's great, but looking at components one-by-one can get boring fast.

All InfrastructureModels packages provide a `print_summary` function that prints a table-like summary of the network data to the terminal.

In [6]:
GasModels.print_summary(data)

Metadata
  R: 8.314
  baseP: 8101325
  baseQ: 604.167
  compressibility_factor: 0.800
  gas_molar_mass: 0.019
  gas_specific_gravity: 0.600
  multinetwork: false
  name: gaslib40
  per_unit: true
  sound_speed: 312.805
  source_type: .m
  source_version: 0.0.0+
  specific_heat_capacity_ratio: 1.400
  standard_density: 1.000
  temperature: 273.150

Table Counts
  junction: 46
  connection: 51
  producer: 3
  consumer: 29


Table: junction
          junction_i,  pmin,  pmax,  p_nominal
       0:          0, 0.013, 1.000, 101325.000
       1:          1, 0.383, 1.000, 3.101325e6
       2:          2, 0.383, 1.000, 3.101325e6
       3:          3, 0.013, 1.000, 101325.000
       4:          4, 0.013, 1.000, 101325.000
       5:          5, 0.383, 1.000, 3.101325e6
       6:          6, 0.013, 1.000, 101325.000
       7:          7, 0.013, 1.000, 101325.000
       8:          8, 0.013, 1.000, 101325.000
       9:          9, 0.013, 1.000, 101325.000
      10:         10, 0.013, 1.000, 10132

Dictionary comprehensions also provide an easy way to take slices of the data.

For example lets inspect the generation profile and output capabilities using comprehensions.

In [8]:
[gen["pg"] for (i,gen) in data["gen"]]

5-element Array{Float64,1}:
 1.0 
 0.2 
 3.0 
 0.85
 2.6 

In [9]:
[gen["pg"] + gen["qg"]im for (i,gen) in data["gen"]]

5-element Array{Complex{Float64},1}:
  1.0 + 0.0im
  0.2 + 0.0im
  3.0 + 0.0im
 0.85 + 0.0im
  2.6 + 0.0im

In [10]:
[(gen["qmin"], gen["qmax"]) for (i,gen) in data["gen"]]

5-element Array{Tuple{Float64,Float64},1}:
 (-1.5, 1.5)    
 (-0.3, 0.3)    
 (-4.5, 4.5)    
 (-1.275, 1.275)
 (-3.9, 3.9)    

### Solving Optimal Power Flow (OPF)

Before we can solve a power model, we need a solver.  Lets start with the simple linear programming solver GLPK and try to solve a DC OPF model.

In [11]:
using GLPKMathProgInterface
lp_solver = GLPKSolverLP()

result = run_dc_opf(data, lp_solver)

Dict{String,Any} with 8 entries:
  "solve_time"   => 2.10108
  "status"       => :Optimal
  "objective"    => 17479.9
  "solver"       => "GLPKMathProgInterface.GLPKInterfaceLP.GLPKSolverLP"
  "solution"     => Dict{String,Any}("baseMVA"=>100.0,"gen"=>Dict{String,Any}("…
  "data"         => Dict{String,Any}("name"=>"pglib_opf_case5_pjm","bus_count"=…
  "machine"      => Dict("cpu"=>"Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz","me…
  "objective_lb" => -Inf

The result object contains a variety of useful information about the optimization problem solved, including the objective value and wall clock runtime.  Detailed documentation of the result dictionary is available [here](https://lanl-ansi.github.io/PowerModels.jl/stable/result-data/), however most interesting point is the `solution` data.

In [12]:
result["solution"]

Dict{String,Any} with 4 entries:
  "baseMVA"  => 100.0
  "gen"      => Dict{String,Any}("4"=>Dict{String,Any}("qg"=>NaN,"pg"=>0.0),"1"…
  "bus"      => Dict{String,Any}("4"=>Dict{String,Any}("va"=>0.0,"vm"=>1),"1"=>…
  "per_unit" => true

Not every useful, lets try `print_summary`,

In [13]:
PowerModels.print_summary(result["solution"])

Metadata
  baseMVA: 100.000
  per_unit: true

Table Counts
  bus: 5
  gen: 5


Table: bus
             va
      1:  0.057
      2: -0.014
      3: -0.008
      4:  0.000
      5:  0.072

  default values:
    vm: 1


Table: gen
            pg
      1: 0.400
      2: 1.700
      3: 3.235
      4: 0.000
      5: 4.665

  default values:
    qg: NaN0000


Looks great!  But `vm` and `qg` are missing...  On to AC OPF!

In [14]:
result = run_ac_opf(data, lp_solver)

MethodError: MethodError: no method matching NonlinearModel(::GLPKMathProgInterface.GLPKInterfaceLP.GLPKSolverLP)
Closest candidates are:
  NonlinearModel(!Matched::Int64) at /Users/carleton/.julia/packages/MathProgBase/rVlFR/src/SolverInterface/SolverInterface.jl:29
  NonlinearModel() at /Users/carleton/.julia/packages/MathProgBase/rVlFR/src/SolverInterface/SolverInterface.jl:27

#### Quiz Question
That was ugly... What happened?!?


Lets try using an NLP solver.

In [15]:
using Ipopt
nlp_solver = IpoptSolver(print_level=0)

result = run_ac_opf(data, nlp_solver)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



Dict{String,Any} with 8 entries:
  "solve_time"   => 4.20517
  "status"       => :LocalOptimal
  "objective"    => 17551.9
  "solver"       => "IpoptSolver"
  "solution"     => Dict{String,Any}("baseMVA"=>100.0,"gen"=>Dict{String,Any}("…
  "data"         => Dict{String,Any}("name"=>"pglib_opf_case5_pjm","bus_count"=…
  "machine"      => Dict("cpu"=>"Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz","me…
  "objective_lb" => -Inf

It worked!  Lets look at the solution,

In [16]:
PowerModels.print_summary(result["solution"])

Metadata
  baseMVA: 100.000
  per_unit: true

Table Counts
  bus: 5
  gen: 5


Table: bus
            vm,     va
      1: 1.078,  0.049
      2: 1.084, -0.013
      3: 1.100, -0.010
      4: 1.064,  0.000
      5: 1.069,  0.063


Table: gen
            pg,     qg
      1: 0.400,  0.300
      2: 1.700,  1.275
      3: 3.245,  3.900
      4: 0.000, -0.108
      5: 4.707, -1.650


The solution make sense intuitively.  The voltage magnitude on bus 3 is pushed to the upper limit, presumably to minimize line losses.  Total generation costs are similar to DC-OPF which seems reasonable.

The solution seems reasonable, but is it optimal?  To test that we need a convex relaxation!  But first a brief digression.

A little secret, the functions `run_dc_opf` and `run_ac_opf` are short hands for a much more general `run_opf` function which supports many variants of optimal power flow problem.  Specifically,
```
run_dc_opf(data, solver) => run_opf(data, DCPPowerModel, solver)
run_dc_opf(data, solver) => run_opf(data, ACPPowerModel, solver)
```

Try the following code with different PowerModels.  Suggested models include `ACPPowerModel`,`ACRPowerModel`,`DCPPowerModel`,`DCPLLPowerModel`,`LPACCPowerModel`, `SOCWRPowerModel`, `QCWRPowerModel`.  A fairly complete list of implemented models is available [here](https://lanl-ansi.github.io/PowerModels.jl/stable/formulation-details/).

In [17]:
result = run_opf(data, ACPPowerModel, nlp_solver)
PowerModels.print_summary(result["solution"])
println()
println("Objective: $(result["objective"])")

Metadata
  baseMVA: 100.000
  per_unit: true

Table Counts
  bus: 5
  gen: 5


Table: bus
            vm,     va
      1: 1.078,  0.049
      2: 1.084, -0.013
      3: 1.100, -0.010
      4: 1.064,  0.000
      5: 1.069,  0.063


Table: gen
            pg,     qg
      1: 0.400,  0.300
      2: 1.700,  1.275
      3: 3.245,  3.900
      4: 0.000, -0.108
      5: 4.707, -1.650

Objective: 17551.890899394704


At this point you might be curious what is PowerModels doing under the hood.  In short, it builds and solves JuMP models.  The [PowerModelsAnnex](https://github.com/lanl-ansi/PowerModelsAnnex.jl/tree/master/src/model) includes some examples of the kind of JuMP models that PowerModels when you call `run_opf`.

Another way to see what PowerModels is doing is to pass your own JuMP model, then you can inspect it later.

In [18]:
using JuMP

m = Model()
result = run_opf(data, DCPPowerModel, nlp_solver, jump_model=m)

println(m)

Min 4000 0_1_pg[4] + 1500 0_1_pg[2] + 3000 0_1_pg[3] + 1000 0_1_pg[5] + 1400 0_1_pg[1]
Subject to
 0_1_va[4] = 0
 0_1_p[(6, 4, 5)] - 0_1_p[(2, 1, 4)] - 0_1_p[(5, 3, 4)] - 0_1_pg[4] = -4
 0_1_p[(4, 2, 3)] - 0_1_p[(1, 1, 2)] = -3
 0_1_p[(5, 3, 4)] - 0_1_p[(4, 2, 3)] - 0_1_pg[3] = -3
 -0_1_p[(3, 1, 5)] - 0_1_p[(6, 4, 5)] - 0_1_pg[5] = 0
 0_1_p[(2, 1, 4)] + 0_1_p[(3, 1, 5)] + 0_1_p[(1, 1, 2)] - 0_1_pg[2] - 0_1_pg[1] = 0
 0_1_p[(4, 2, 3)] - 91.67583425009167 0_1_va[2] + 91.67583425009167 0_1_va[3] = 0
 0_1_va[2] - 0_1_va[3] ≤ 0.5235987755982988
 0_1_va[2] - 0_1_va[3] ≥ -0.5235987755982988
 0_1_p[(2, 1, 4)] - 32.56904637832204 0_1_va[1] + 32.56904637832204 0_1_va[4] = 0
 0_1_va[1] - 0_1_va[4] ≤ 0.5235987755982988
 0_1_va[1] - 0_1_va[4] ≥ -0.5235987755982988
 0_1_p[(3, 1, 5)] - 154.7029702970297 0_1_va[1] + 154.7029702970297 0_1_va[5] = 0
 0_1_va[1] - 0_1_va[5] ≤ 0.5235987755982988
 0_1_va[1] - 0_1_va[5] ≥ -0.5235987755982988
 0_1_p[(5, 3, 4)] - 33.33666700003334 0_1_va[3] + 33.33666700003334

### Computing Optimality Gaps

Getting back to the question of AC-OPF solution quality.  An easy way to certify AC-OPF solution quality is to use a convex-relaxation to compute a lower bound.  If the "gap" between the solution and the lower bound is small we are happy.

Let build a helper function for computing optimality gaps.

In [19]:
function soc_gap(data_file, nlp_solver)
    data = PowerModels.parse_file(data_file)
    ac_result = run_opf(data, ACPPowerModel, nlp_solver)
    soc_result = run_opf(data, SOCWRPowerModel, nlp_solver)
    
    # it is important to check the solver terminated nominally!
    @assert  ac_result["status"] == :Optimal ||  ac_result["status"] == :LocalOptimal
    @assert soc_result["status"] == :Optimal || soc_result["status"] == :LocalOptimal
    
    # by convention optimality gaps are given as a percentage
    gap = 100*(ac_result["objective"] - soc_result["objective"])/ac_result["objective"]
    
    return (gap=gap, ub_time=ac_result["solve_time"], lb_time=soc_result["solve_time"]) 
end

soc_gap (generic function with 1 method)

In [20]:
soc_gap("data/pglib_opf_case5_pjm.m", nlp_solver)

(gap = 14.540746218458999, ub_time = 0.019286024, lb_time = 0.025523464)

A 14% gap, that's HUGE!  It is not clear if the AC-OPF solutoin is optimal or not.  Clearly this case requires more study.

In the mean time, try the `soc_gap` function on a few more of the network cases included with this tutorial.  Available datasets are `pglib_opf_case14_ieee.m`, `pglib_opf_case30_ieee.m`, `pglib_opf_case73_ieee_rts.m`, `pglib_opf_case118_ieee.m`, `pglib_opf_case200_tamu.m`, `pglib_opf_case500_tamu.m`, `pglib_opf_case1354_pegase.m`, `pglib_opf_case2383wp_k.m` (larger may take a few minutes to run).  The complete PGLib-OPF benchmark library with over 100 cases is available [here](https://github.com/power-grid-lib/pglib-opf).

In [21]:
soc_gap("data/pglib_opf_case14_ieee.m", nlp_solver)

(gap = 0.10909416417088194, ub_time = 0.036981832, lb_time = 0.027105066)

#### Discussion Point

Can you draw any conclusions from computing optimality gaps on a variety of cases?


### A Simple Contingency Study

Component `status` parameters can be useful for exploring a variety of component failure scenarios.  Lets use it to do a simple generator N-1 study on the 5-bus example.


In [22]:
function single_generator_loss(data_file, nlp_solver)
    data = PowerModels.parse_file(data_file)
    
    result = run_opf(data, ACPPowerModel, nlp_solver)

    # it is important to check the solver terminated nominally!
    @assert  result["status"] == :Optimal || result["status"] == :LocalOptimal

    println("base cost: $(result["objective"])")
    
    for (i,gen) in sort(data["gen"])
        # remove the generator
        gen["gen_status"] = 0
        
        result = run_opf(data, ACPPowerModel, nlp_solver)
        
        println("gen $(i) out: $(result["objective"]), $(result["status"])")

        # restore the generator
        gen["gen_status"] = 1
    end
end

single_generator_loss (generic function with 1 method)

In [23]:
single_generator_loss("data/pglib_opf_case5_pjm.m", nlp_solver)

base cost: 17551.890899394704
gen 1 out: 17680.162671442777, LocalOptimal
gen 2 out: 17929.212321096078, LocalOptimal
gen 3 out: 0, LocalInfeasible
gen 4 out: 17553.565008204587, LocalOptimal


┌ Warning: Not solved to optimality, status: Infeasible
└ @ JuMP /Users/carleton/.julia/packages/JuMP/PbnIJ/src/nlp.jl:1283


gen 5 out: 0, LocalInfeasible


┌ Warning: Not solved to optimality, status: Infeasible
└ @ JuMP /Users/carleton/.julia/packages/JuMP/PbnIJ/src/nlp.jl:1283


This simple study seems to suggest that this network is not N-1 secure and cannot support the loss of generator 3 or 5.  Let's compute how much these two generators contribute to the total generation capacity in the data set.

In [24]:
pg_capacity = sum(gen["pmax"] for (i,gen) in data["gen"])

15.3

In [25]:
100*data["gen"]["3"]["pmax"]/pg_capacity

33.98692810457516

In [26]:
100*data["gen"]["5"]["pmax"]/pg_capacity

39.2156862745098

It seems reasonable that losing over 30% of the total generating capacity would be problematic.

### Other Topics

PowerModels includes a variety of useful features that are outside the scope of this brief tutorial.  These features include,
* TBD